In [3]:
import requests
import json
import os
import logging

# You can also log to the console by using the StreamHandler (optional)
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)  # Display INFO level and above in the console
console_handler.setFormatter(logging.Formatter("%(asctime)s - %(levelname)s - %(message)s"))
logging.getLogger().addHandler(console_handler)


In [4]:
API_URL = "http://api.jolpi.ca/ergast/f1/2024/21/results.json"

def fetch_race_results():
    """Fetch the F1 race results for the specified round."""
    logging.info("Starting to fetch race results from API...")
    try:
        response = requests.get(API_URL)
        response.raise_for_status()  # Check if request was successful
        data = response.json()
        logging.info("Successfully fetched race results.")
        
        # Extracting only the results from the data
        results = data['MRData']['RaceTable']['Races'][0]['Results']
        return results
    except Exception as e:
        logging.error(f"Error fetching data: {e}")
        return None


In [5]:
PICKS_FILE = "player_picks.json"

def save_picks(picks):
    """Save player picks locally to a JSON file."""
    try:
        logging.info(f"Saving picks for {len(picks)} players.")
        with open(PICKS_FILE, 'w') as file:
            json.dump(picks, file)
        logging.info("Picks saved successfully.")
    except Exception as e:
        logging.error(f"Error saving picks: {e}")

def load_picks():
    """Load player picks from the JSON file."""
    if os.path.exists(PICKS_FILE):
        try:
            with open(PICKS_FILE, 'r') as file:
                picks = json.load(file)
            logging.info("Picks loaded successfully.")
            return picks
        except Exception as e:
            logging.error(f"Error loading picks: {e}")
            return {}
    logging.info("No previous picks found.")
    return {}


In [6]:
def get_player_picks():
    """Prompt user for their driver picks."""
    picks = {}
    num_players = int(input("Enter number of players: "))
    logging.info(f"Number of players: {num_players}")
    for i in range(num_players):
        player = input(f"Enter Player {i+1}'s name: ")
        drivers = input(f"Enter {player}'s driver picks (comma-separated codes): ").split(',')
        picks[player] = [d.strip().upper() for d in drivers]
        logging.info(f"Player {player} picked drivers: {', '.join(picks[player])}")
    save_picks(picks)
    return picks


In [7]:
POINTS = {1: 25, 2: 18, 3: 15, 4: 12, 5: 10, 6: 8, 7: 6, 8: 4, 9: 2, 10: 1}

def calculate_points(results, picks):
    """Calculate points for each player based on race results and picks."""
    logging.info("Calculating points for each player...")
    driver_points = {}
    for result in results:
        position = int(result['position'])
        code = result['Driver']['code']
        driver_points[code] = POINTS.get(position, 0)

    player_scores = {}
    for player, drivers in picks.items():
        score = sum(driver_points.get(driver, 0) for driver in drivers)
        player_scores[player] = score
        logging.info(f"Player {player} scored {score} points.")
    return player_scores


In [8]:
def main():
    """Main function to fetch results, get picks, and calculate scores."""
    logging.info("Program started.")
    
    # Load or prompt for player picks
    picks = load_picks()
    if not picks:
        logging.info("No picks found, prompting for player picks.")
        picks = get_player_picks()

    # Fetch race results
    data = fetch_race_results()
    if data:
        try:
            logging.info("Race results fetched successfully.")
            results = data
            scores = calculate_points(results, picks)

            # Display results
            logging.info("Displaying fantasy scores.")
            print("\nFantasy F1 Scores:")
            for player, score in scores.items():
                print(f"{player}: {score} points")
        except KeyError as e:
            logging.error(f"Error parsing data: {e}")
    else:
        logging.error("No data fetched.")
        print("No data fetched.")


In [9]:
main()


Fantasy F1 Scores:
Ben: 25 points
